In [1]:
# the utils we'll be looking for

# we created groups
# we could calculate intersections
# we could infill groups with -1

# where is my interpolate and combine?

# the interval type seems realy handy as a multiindex with a float value field
    # multi type sleep data
    # single type data present/ not present  for various sensors
    # action intervals with various string descriptors

# is there a way for me to have multiple kinds of values for the value segment?

# start with the coded int intervals
    #how to keep the key? - let's just do it in the docs for now
# then extend to the action intervals

# how about the key for the coded int is the column name
#name+int!....

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import os
import sys

cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, writeWorkingTSDf


In [3]:
def infillIntervals(intervals,col_name):
    # Shift columns to compare consecutive rows
    next_start = intervals['sampleDT'].shift(-1)

    # Find rows where there is a gap
    gap_mask = next_start > intervals['endTime']

    # Create a DataFrame for the gaps
    gap_df = pd.DataFrame({
        'sampleDT': intervals.loc[gap_mask, 'endTime'],
        'endTime': next_start[gap_mask],
        col_name: 0
    })

    # Concatenate the original DataFrame and the gap DataFrame
    combined_df = pd.concat([intervals, gap_df])

    return combined_df.sort_values('sampleDT').reset_index(drop=True)

In [4]:
#modify the dd datatype to append "-dataIntervals-86400"
# in seconds with p for .


# so something we can do is make intervals for existing data
# min and max interval are in seconds
def dataIntervals(dd, max_interval):
    #take in a device descriptor and min and max interval in seconds?
    #column name would be noData+0!data+1
    tsDf = readWorkingTSDF(dd)

    time_diffs = tsDf.index.to_series().diff()

    # Mark where the time difference exceeds the threshold (group break points)
    group_breaks = time_diffs.dt.total_seconds() > max_interval

    # Assign group labels by cumulatively summing the group breaks
    group_labels = group_breaks.cumsum()

    # Create a dataframe of (start_time, end_time) for each group
    group_labels_times = pd.Series(group_labels.index, index=group_labels.values)
    gStarts = group_labels_times[~group_labels_times.index.duplicated(keep="first")]
    gEnds = group_labels_times[~group_labels_times.index.duplicated(keep="last")]
    allGroupsDf = pd.DataFrame({'sampleDT':gStarts, 'endTime': gEnds})

    #check min group time
    minGroupTime = pd.Timedelta(seconds=0)
    groupsDf = allGroupsDf[allGroupsDf['endTime'] - allGroupsDf['sampleDT'] > minGroupTime].copy()

    col_name = 'int!noData+0!data+1'
    groupsDf[col_name] = 1
    return infillIntervals(groupsDf, col_name)


In [5]:
#alright new plan for the intervals just use writeWorking TSDF 
# add dedupe = false if needed

In [6]:
testdd = ["abhik", '0', "apple", 'AppleWatch-10-46mm', 'hr', 'builtin']
testdf = readWorkingTSDF(testdd)

read in 1503602 rows from 3 files, retruning 1503602 rows


In [7]:
intervs = dataIntervals(testdd, 11)

read in 1503602 rows from 3 files, retruning 1503602 rows


In [8]:
intervs


,sampleDT,endTime,int!noData+0!data+1
0,2024-09-28 10:00:33+00:00,2024-09-28 11:52:23+00:00,1
1,2024-09-28 11:52:23+00:00,2024-09-28 11:52:35+00:00,0
2,2024-09-28 11:52:35+00:00,2024-09-28 13:20:52+00:00,1
3,2024-09-28 13:20:52+00:00,2024-09-28 13:21:06+00:00,0
4,2024-09-28 13:21:06+00:00,2024-09-28 15:04:07+00:00,1
...,...,...,...
12820,2025-01-14 17:11:54+00:00,2025-01-14 17:12:56+00:00,1
12821,2025-01-14 17:12:56+00:00,2025-01-14 17:14:10+00:00,0
12822,2025-01-14 17:14:10+00:00,2025-01-14 19:06:16+00:00,1
12823,2025-01-14 19:06:16+00:00,2025-01-14 19:06:38+00:00,0


In [17]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta
import pytz

def plot_intervals_with_plotly(intervals, year, month, week, timezone='UTC'):
    # Calculate the start and end dates for the specified week
    first_day_of_month = datetime(year, month, 1, tzinfo=pytz.timezone(timezone))
    start_of_week = first_day_of_month + timedelta(weeks=week-1)
    end_of_week = start_of_week + timedelta(days=7)

    # Filter intervals for the specified week
    week_intervals = intervals[
        (intervals['sampleDT'] >= start_of_week) & 
        (intervals['sampleDT'] < end_of_week)
    ]

    # Prepare data for plotting
    times = []
    values = []
    for _, row in week_intervals.iterrows():
        times.extend([row['sampleDT'], row['endTime']])
        values.extend([row['int!noData+0!data+1'], row['int!noData+0!data+1']])

    # Create the plot
    fig = go.Figure()

    # Add line plot
    fig.add_trace(go.Scatter(
        x=times,
        y=values,
        mode='lines+markers',
        name='Interval',
        line=dict(width=1),
        marker=dict(size=4)
    ))

    # Update layout
    fig.update_layout(
        title=f"Intervals for {year}-{month:02d}, Week {week}",
        xaxis_title="Time",
        yaxis_title="Value",
        xaxis=dict(
            range=[start_of_week, end_of_week],
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        yaxis=dict(
            tickvals=[-1, 0, 1, 2, 3],
            ticktext=["No Data", "Awake", "Light", "Deep", "REM"]
        )
    )

    # Show the plot
    fig.show()

# Example usage
year = 2025
month = 1
week = 1
plot_intervals_with_plotly(intervs, year, month, week)

In [18]:
dict(count=1, label="1d", step="day", stepmode="backward")

{'count': 1, 'label': '1d', 'step': 'day', 'stepmode': 'backward'}

In [15]:
# what did I want to do with these intervals?
# determine where intervals for audio data were so I could query the aligned data struture better for actual audio

# query times when cameras were recording fast, slow, or not at all

# query intersecting HR intervals

# store hypnographs

# find gaps in air quality data

# TODO make separate read and write interval functions

In [16]:
# after I get my audio imported and intervaled
# pandas allows for non unique indicies 
# just don't deduplicate the data
# we can store the activites in a standard interval 